# Focused Ion Beam Molecular Dynamics (fibmd) Tool

In [43]:
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
#from ipywidgets import Button, Layout
from string import Template
import moviepy.editor as mpy
import hublib.use
from hublib.cmd import runCommand
from hublib.ui import FileUpload, Download
from IPython.display import FileLink, FileLinks
from hubzero.submit.SubmitCommand import SubmitCommand

In [44]:
%use openmpi-1.6.3-gnu-4.7.2

# Creating widgets for critical input

In [45]:
#title = widgets.Label(value="FIBMD Tool", width=10)

box_layout = widgets.Layout(display='flex',
                    jusitf='center'
                   )
keVin = widgets.BoundedFloatText(
    value=2000,
    min=1000,
    max=60000,
    step=100
)

xdimin = widgets.BoundedIntText(
    value=6,
    min=4,
    max=40,
    step=1,
    description='X:'
)

ydimin = widgets.BoundedIntText(
    value=6,
    min=4,
    max=40,
    step=1,
    description='Y: '
)

zdimin = widgets.BoundedIntText(
    value=4,
    min=1,
    max=40,
    step=1,
    description='Z:'
)

pin = widgets.BoundedIntText(
    value = 1,
    min = 1,
    max = 10,
    step = 1
)
nin = widgets.BoundedIntText(
    value = 1,
    min = 1,
    max = 100,
    step = 1
)

ain  = widgets.BoundedIntText(
    value = 500,
    min = 0,
    max = 1e6,
    step = 100
)

din = widgets.FloatText(
    value = 1,
    step = 0.1
)

phizin = widgets.FloatText(
    value = 0.0,
    step = 1
)
phixyin = widgets.FloatText(
    value = 0.0,
    step = 1
)

fin = widgets.FloatText(
    value = 2
)

dimlabel = widgets.Label(" Target dimensions (in silicon unit cells)",layout=box_layout)
keVlabel = widgets.Label("Ion Energy:")
proclabel = widgets.Label("# of Processors:")
ionlabel = widgets.Label("# of Ions to fire:")
outlabel = widgets.Label(".xyz output frequency:")
dtlabel = widgets.Label("Time between ion impacts in ps:")
anglezlabel = widgets.Label("Angle of incidence (from a-axis):")
anglexylabel = widgets.Label("Angle of incidence (from x-axis, xy plane):")
fwhmlabel = widgets.Label("FWHM of ion beam in nm:")

xcol = widgets.VBox([widgets.Label(" "),xdimin])
ycol = widgets.VBox([dimlabel,ydimin],layout=box_layout)
zcol = widgets.VBox([widgets.Label(" "),zdimin])

dimensions = widgets.HBox([xcol,ycol,zcol])

vlabels = widgets.VBox([keVlabel, proclabel, ionlabel, outlabel, dtlabel, anglezlabel, anglexylabel, fwhmlabel])
vinput = widgets.VBox([keVin, pin, nin, ain, din, phizin, phixyin, fin])

inputs2 = widgets.HBox([vlabels,vinput])

#phizin2 = widgets.HBox([widgets.Label("Angle of Incidence (from z-axis)"),phizin1])
#phixyin2 = widgets.HBox([widgets.Label("Angel of Incidence (from x-axis, xy plane)"),phixyin1])

fwhmin = widgets.HBox([widgets.Label("FWHM of ion beam in nm:"),fin])


In [46]:
widgets.VBox([dimensions,inputs2],layout=widgets.Layout(border='solid'))

VBox(children=(HBox(children=(VBox(children=(Label(value=' '), BoundedIntText(value=6, description='X:', max=40, min=4))), VBox(children=(Label(value=' Target dimensions (in silicon unit cells)', layout=Layout(display='flex')), BoundedIntText(value=6, description='Y: ', max=40, min=4)), layout=Layout(display='flex')), VBox(children=(Label(value=' '), BoundedIntText(value=4, description='Z:', max=40, min=1))))), HBox(children=(VBox(children=(Label(value='Ion Energy:'), Label(value='# of Processors:'), Label(value='# of Ions to fire:'), Label(value='.xyz output frequency:'), Label(value='Time between ion impacts in ps:'), Label(value='Angle of incidence (from a-axis):'), Label(value='Angle of incidence (from x-axis, xy plane):'), Label(value='FWHM of ion beam in nm:'))), VBox(children=(BoundedFloatText(value=2000.0, max=60000.0, min=1000.0, step=100.0), BoundedIntText(value=1, max=10, min=1), BoundedIntText(value=1, min=1), BoundedIntText(value=500, max=1000000, step=100), FloatText(value=1.0, step=0.1), FloatText(value=0.0, step=1.0), FloatText(value=0.0, step=1.0), FloatText(value=2.0)))))), layout=Layout(border='solid'))

In [47]:
runbut = widgets.Button(description='Run Simulation')
display(runbut)

Button(description='Run Simulation', style=ButtonStyle())

submit --parameters --local --tail out.log mpirun -np 1 bin/mdrun2 > out.log
 ----------------Parallel Environment Initialized, 
           1  processors are alive and running ....-------------------
 Initparallel done
 READING PARAMETERS
 Si atoms        1152
   3.2586437999999998E-009
   3.2586437999999998E-009
   2.1724291999999999E-009
 called ionrun           1
 readprms done
 initran done
 initdata done
 initparaops done
 init_nlist done
 initio done
 mktables done
 initmeans done
 si_nlist done
 DONE INITIALIZING
 --------------------------------
         0    0.1173337040E+02    0.4613962044E+02
        50    0.1930247586E+03   -0.6572071544E-04
       100    0.2588448461E+03    0.4796569859E-03
 Equalized to temperature    270.34439258422060     
Total time for      0th impact is ********** seconds
-----------------------------------------------------------------------------
| at_time    0.1051425934E-03 at_count        2 average    0.5257129669E-04 |
| nt_time    0.9342908859

mpirun: killing job...

mpirun: abort is already in progress...hit ctrl-c again to forcibly terminate



In [48]:
def get_template():
    if ain.value == 0:
        atmoutin1 = -1
        atmoutin2 = -1
    else:
        atmoutin1 = int(ain.value/10)
        atmoutin2 = ain.value
    inputs = list([str(keVin.value),str(xdimin.value),str(ydimin.value),str(zdimin.value),
                         str(pin.value),str(1),str(1),str(nin.value),str(atmoutin1),str(atmoutin2),
                         str(din.value*1e-12),str(fin.value),str(phizin.value),str(phixyin.value)])
    tags = list(['keV','xdim','ydim','zdim',
                 'procsx','procsy','procsz','nlj','atmout1','atmout2',
                 'dtion','fwhm','phiz','phixy'])

    input_dict = dict(zip(tags, inputs))
    temp_contents = open('siga.in.template').read()
    tempstr = Template(temp_contents)
    print(tempstr)
    # Python template strings replace "${identifier}" with the value.
    # If substituting more than one value, use a dictionary.
    return tempstr.substitute(input_dict)

In [49]:
def run_mdrun2():
    with open('siga.in', 'w') as tfile:
        tfile.write(get_template())
        
    submitCommand = SubmitCommand()
    submitCommand.setParameters('--local')
#    submitCommand.setTailStdout(tailStdout=True, tailStdoutNlines=20)
#    submitCommand.setTailFiles('out.log')
#    submitCommand.setDetach(detach=True)    
    submitCommand.setCommand('mpirun -np %i bin/mdrun2 > out.log'%(pin.value))
    submitCommand.show()
    submitCommand.submit()
#    res, stdout, stderr = runCommand('mpirun -np %i bin/mdrun2'%(pin.value))
#    res, stdout, stderr = runCommand('submit --local --detach mpirun -np %i bin/mdrun2 > out.log'%(pin.value))
#    SubmitCommand.submit(['--local','--detach','mpirun -np %i bin/mdrun2 > out.log'%(pin.value)])
    #!mpirun -np $procsin.value src/mdrun2 #> out.log | tail -10 out.log #> out.log# -10 out.log
    return

In [50]:
# This is the main function called when the Run button is clicked.
def run_simulation(ignore):
    # change label on Button
    runbut.disabled=True
    runbut.description='Running'

    out2 = run_mdrun2()

    # Change button label back and re-enable
    runbut.description='Run Simulation'
    runbut.disabled=False
    

out1 = runbut.on_click(run_simulation)

In [64]:
#file download links
outfile = FileLink("out.log")
display(local_file)
xyzfile = FileLink("data/mdrun2.xyz")
display(xyzfile)

/home/nanohub/jmstout2/notebooks/fib_md/out.log

/home/nanohub/jmstout2/notebooks/fib_md/data/mdrun2.xyz

In [53]:

#requires detaching from the submit command, which is not working 

##This is a function call for refreshing an output box from the 
#tailbut = widgets.Button(description = "Refresh Text Output")
#display(tailbut)

#def check_output(ignore):
#    clear_output()
#    reso,stdouto,stderro = runCommand('tail --lines 30 -f out.log')
    
#out3 = tailbut.on_click(check_output)